### Reference.
[Torch Logistic Regression Demo](https://github.com/torch/demos/blob/master/logistic-regression/example-logistic-regression.lua)

In [1]:
require 'nn'
require 'optim'
require 'csvigo'

In [2]:
-- 1. Create Training Data

-- Load Data
data = csvigo.load('example-logistic-regression.csv')

brands = torch.Tensor(data.brand)    -- Target
females = torch.Tensor(data.female)
ages = torch.Tensor(data.age)

<csv>	parsing file: example-logistic-regression.csv	


<csv>	tidying up entries	
<csv>	returning tidy table	


In [3]:
-- Copy all the variable into a single tensor
dataSet = torch.Tensor( (#brands)[1],2 )
dataSet[{ {}, 1 }] = females
dataSet[{ {}, 2 }] = ages

targets = brands

In [4]:
numOfBrand = 3  -- 1,2,3

-- Summarize the Data
print("Number of brands : ".. numOfBrand)

print("Max brand : ".. torch.max(brands))
print("Min brand : ".. torch.min(brands))

print("Max female : ".. torch.max(females))
print("Min female : ".. torch.min(females))

print("Max age : ".. torch.max(ages))
print("Min age : ".. torch.min(ages))

print("dataSet Shape.")
print(#dataSet)

Number of brands : 3	
Max brand : 3	
Min brand : 1	
Max female : 1	
Min female : 0	
Max age : 38	
Min age : 24	
dataSet Shape.	
 735
   2
[torch.LongStorage of size 2]



In [5]:
trainX = dataSet[{ {1,700} }]
trainY = targets[{ {1,700} }]
testX = dataSet[{ {701,735} }]
testY = targets[{ {701,735} }]

print(#trainX)
print(#trainY)
print(#testX)
print(#testY)

 700
   2
[torch.LongStorage of size 2]

 700
[torch.LongStorage of size 1]

 35
  2
[torch.LongStorage of size 2]

 35
[torch.LongStorage of size 1]



In [6]:
-- 2. Define the Model

model = nn.Sequential()
model:add(nn.Linear(2,3))
model:add(nn.LogSoftMax())

In [7]:
-- 3. Loss and Optimizer

criterion = nn.ClassNLLCriterion() -- Log-likelihood(NLL)

In [8]:
-- 4. Train the model

x, dW = model:getParameters()

feval = function(x_new)
    if x ~= x_new then
        x:copy(x_new)
    end
    
    _nidx_ = (_nidx_ or 0) + 1
    if _nidx_ > (#trainX)[1] then
        _nidx_ = 1
    end
    
    local inputs = trainX[_nidx_]
    local target = trainY[_nidx_]
    
    -- Reset Gradient
    dW:zero()
    
    local loss = criterion:forward(model:forward(inputs), target)
    model:backward(inputs, criterion:backward(model.output, target))
    
    return loss, dW
end

In [9]:
sgd_params = {
    learning_rate = 1e-3,
    learningRateDecay = 1e-4,
    weightDecay = 0,
    momentum = 0
}

numIters = 100

In [10]:
-- Training with SGD
for i =1, numIters do
    current_loss = 0
    -- Epoch
    for i = 1, (#trainX)[1] do
        _, loss = optim.sgd(feval, x, sgd_params)
        current_loss = current_loss + loss[1]
    end
    
    -- Report average error on Epoch
    if i%10 == 0 then
        current_loss = current_loss / (#trainX)[1]
        print("Epoch : " .. i .. " Loss=" .. current_loss)
    end
end

Epoch : 10 Loss=1.0885289198155	


Epoch : 20 Loss=1.0557149100252	


Epoch : 30 Loss=1.0393850989384	


Epoch : 40 Loss=1.0298807257468	


Epoch : 50 Loss=1.0239110556529	


Epoch : 60 Loss=1.0200314196615	


Epoch : 70 Loss=1.0174960449349	


Epoch : 80 Loss=1.0158748419338	


Epoch : 90 Loss=1.0148996776959	


Epoch : 100 Loss=1.0143937714144	


In [11]:
-- 5. Testing the model

function predict(testX)
    local logProbs = model:forward(testX)  
    local probs = torch.exp(logProbs)

    percent, target = torch.max(probs, 1)
    return target[1]
end

In [12]:
accuracy = 0
for i = 1, (#testX)[1] do
    predY = predict(testX[i])
    if predY == testY[i] then
        accuracy = accuracy + 1
    end
end

print(accuracy / (#testX)[1])

0.71428571428571	
